In [122]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from random import gauss
import warnings
warnings.filterwarnings('ignore')

In [123]:
with open('/Users/Dovla/Downloads/quotedata.dat','r') as f:
    underlying = float(f.readline().split(',')[1])
dat = pd.read_csv('/Users/Dovla/Downloads/quotedata.dat', sep=",", skiprows=2)
calls = dat.iloc[:,0:7]
calls['Y'], calls['M'],calls['D'],calls['K'],calls['Code'] = calls['Calls'].str.split(' ', -1).str
calls['Code'],calls['Exch'] = calls['Code'].str.split('-', -1).str
calls['K'] = calls.K.astype(float)
calls['Time'] = pd.to_datetime(calls.Y+calls.M+calls.D,format='%Y%b%d')
calls['Time1'] = calls['Time'] - calls['Time'][0]
calls['Ti'] = calls['Time1'].dt.days / 365
calls['S'] = underlying
calls['r'] = 0.03
calls['opMidPrice'] = (calls['Bid'] + calls['Ask'])/2
calls['sigma'] = 0.1
calls = calls[calls['Exch'].isnull()]
calls1 = calls[['S','K','Ti','r','sigma','opMidPrice']]

In [221]:
def d1(x):
    d1 = (np.log(x.S/x.K) + (x.r + x.sigma**2/2)*x.Ti) / (x.sigma * np.sqrt(x.Ti))
    return d1
def d2(x):
    d2 = x.d1 - x.sigma * np.sqrt(x.Ti)
    return d2
def call(x):
    c0 =  x.S * norm.cdf(x.d1) - x.K * exp(-x.r*(x.Ti)) * norm.cdf(x.d2)
    return c0
def deltaCall(x):
    return norm.cdf(x.d1)
def gamma(x):
    gamma = norm.pdf(x.d1) / (x.S * x.sigma * sqrt(x.Ti))
    return gamma
def thetaCall(x):
    theta = -(x.S*norm.pdf(x.d1)*x.sigma / (2*sqrt(x.Ti))) - (0.03 * x.K*exp(-0.03*x.Ti)*norm.cdf(x.d2))
    return theta
def rhoCall(x):
    rho = ((x.Ti) * x.K * exp(-0.03*(x.Ti)) * norm.cdf(x.d2))
    return rho

In [222]:
calls1['d1'] = calls1.apply(d1, axis=1)
calls1['d2'] = calls1.apply(d2, axis=1)
calls1['opBSprice'] = calls1.apply(call, axis=1)
calls1['deltaCall'] = calls1.apply(deltaCall, axis=1)
calls1['gamma'] = calls1.apply(gamma, axis=1)
calls1['thetaCall'] = calls1.apply(thetaCall, axis=1)
calls1['rhoCall'] = calls1.apply(rhoCall, axis=1)

In [224]:
calls1.tail()

,S,K,Ti,r,sigma,opMidPrice,d1,d2,optBSprice,CallMC,opBSprice,deltaCall,gamma,thetaCall,rhoCall
12784,186.8,340.0,2.167123,0.03,0.1,3.885,-3.553103,-3.700315,0.001257,0.0,0.001257,0.000190,0.000026,-0.005620,0.074337
12800,186.8,350.0,2.167123,0.03,0.1,4.200,-3.750014,-3.897226,0.000560,0.0,0.000560,0.000088,0.000013,-0.002716,0.034578
12816,186.8,360.0,2.167123,0.03,0.1,3.125,-3.941378,-4.088589,0.000246,0.0,0.000246,0.000041,0.000006,-0.001291,0.015864
12832,186.8,370.0,2.167123,0.03,0.1,3.575,-4.127497,-4.274709,0.000107,0.0,0.000107,0.000018,0.000003,-0.000605,0.007190
12848,186.8,380.0,2.167123,0.03,0.1,2.410,-4.308653,-4.455865,0.000046,0.0,0.000046,0.000008,0.000001,-0.000280,0.003224


In [225]:
def GBM(x):
    return x.S * exp((x.r - 0.5 * x.sigma**2) * x.Ti + x.sigma * np.sqrt(x.Ti) * gauss(0,1.0))
def MC(x, sims=100):
    payoffs = []
    for i in range(sims):
        S_T = GBM(x)
        payoffs.append(max(0.0,S_T-x.K))
    price = np.exp(-x.r * (x.Ti)) * (sum(payoffs) / float(sims))
    return price
calls1['CallMC'] = calls1.apply(MC, axis=1)

In [218]:
calls2 = calls1.iloc[500:510,]
def call1(S,K,r,sigma,Ti):
    d1 = (np.log(S/K) + (r + sigma**2/2)*Ti) / (sigma * np.sqrt(Ti))
    d2 = d1 - sigma * np.sqrt(Ti)
    c0 =  S * norm.cdf(d1) - K * exp(-r*(Ti)) * norm.cdf(d2)
    return c0, d1
def vega(S,d1,Ti):
    vega = (S * norm.pdf(d1) * np.sqrt(Ti))
    return vega
def impVol(x):
    iters = 100
    epsilon = 1.0e-5
    vol = 0.5
    for i in range(0, iters):
        price, d111 = call1(x.S,x.K,x.r,vol,x.Ti)
        veg = vega(x.S,d111,x.Ti)
        diff = x.opMidPrice - price  # our root
        if (abs(diff) < epsilon):
            return vol
        vol = vol + diff/veg # f(x) / f'(x)
    return vol
calls2['ImpVol'] = calls2.apply(impVol, axis=1)

In [219]:
def mesh_plot2(X,Y,Z):
   fig = plt.figure()
   ax = Axes3D(fig, azim = -29, elev = 50)
   XX,YY,ZZ = make_surf(X,Y,Z)
   ax.plot_surface(XX,YY,ZZ, color = 'white')
   ax.contour(XX,YY,ZZ)
   plt.xlabel("expiry")
   plt.ylabel("strike")  

def make_surf(X,Y,Z):
   XX,YY = np.meshgrid(np.linspace(min(X),max(X),230),np.linspace(min(Y),max(Y),230))
   ZZ = griddata(np.array([X,Y]).T,np.array(Z),(XX,YY), method='linear')
   return XX,YY,ZZ   



,S,K,Ti,r,sigma,opMidPrice,d1,d2,optBSprice,CallMC,ImpVol
8000,186.8,175.0,0.326027,0.03,0.1,20.600,1.342646,1.285548,13.957050,13.968335,0.310812
8016,186.8,180.0,0.326027,0.03,0.1,17.550,0.849276,0.792177,9.759013,9.841492,0.308864
8032,186.8,185.0,0.326027,0.03,0.1,14.750,0.369424,0.312325,6.256255,6.385967,0.305789
8048,186.8,190.0,0.326027,0.03,0.1,12.275,-0.097631,-0.154729,3.628531,3.753996,0.303367
8064,186.8,195.0,0.326027,0.03,0.1,10.050,-0.552552,-0.609651,1.885613,2.164881,0.299865
8080,186.8,200.0,0.326027,0.03,0.1,8.075,-0.995956,-1.053054,0.872888,0.857359,0.295417
8096,186.8,205.0,0.326027,0.03,0.1,6.500,-1.428410,-1.485509,0.359095,0.306423,0.294021
8112,186.8,210.0,0.326027,0.03,0.1,5.200,-1.850442,-1.907541,0.131317,0.146552,0.293339
8128,186.8,215.0,0.326027,0.03,0.1,4.100,-2.262544,-2.319642,0.042781,0.058508,0.292059
8144,186.8,220.0,0.326027,0.03,0.1,3.175,-2.665171,-2.722269,0.012460,0.000000,0.290025
